In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from datetime import datetime
from datetime import timedelta

In [2]:
from bs4 import BeautifulSoup
import undetected_chromedriver as uc

In [3]:
from pymongo.mongo_client import MongoClient
import pymongo

In [4]:
import hashlib
import time
import pandas as pd
from ta.momentum import RSIIndicator, StochRSIIndicator
from ta.trend import SMAIndicator

In [5]:
# url = 'https://tracksino.com/monopoly?period=1week'
# url = 'https://tracksino.com/monopoly?period=3hours'
url = "https://tracksino.com/monopoly?period=1month"
CURRENT_YEAR = 2024


def generate_hash(record):
    record_string = "".join(str(v) for v in record.values())
    return hashlib.sha256(record_string.encode()).hexdigest()


def get_spin_history_data(soup):
    global CURRENT_YEAR
    spin_history = []
    table = soup.find("table", {"id": "spin-history"})
    rows = table.tbody.find_all("tr")

    def convert_to_datetime(date_str):
        data = date_str.strip() + " " + str(CURRENT_YEAR)
        date_obj = datetime.strptime(data, "%b %d @ %H:%M %Y")

        return date_obj

    for j, row in enumerate(rows):
        cols = row.find_all("td")
        data_row = {}

        for i, col in enumerate(cols):
            if i == 0:
                prev_date_obj = (
                    convert_to_datetime(rows[j - 1].find_all("td")[0].text.strip())
                    if j > 0
                    else None
                )

                if (
                    prev_date_obj
                    and convert_to_datetime(col.text.strip()).month == 12
                    and prev_date_obj.month == 1
                ):
                    CURRENT_YEAR -= 1
                    print("Year changed to:", CURRENT_YEAR)

                date_obj = convert_to_datetime(col.text.strip())

                if date_obj == prev_date_obj:
                    date_obj -= timedelta(seconds=1)

                data_row["date"] = date_obj

            elif i == 1:
                center = col.find("center")
                icon = center.find("i")
                if icon:
                    number = icon["class"][0].split("-")[-1]
                    data_row["number"] = number
            elif i in [2, 3, 4, 5, 6]:
                cell_value = col.text.strip()
                data_row[f"col_{i}"] = (
                    int("".join(filter(str.isdigit, cell_value))) if cell_value else 0
                )
            else:
                break

        if data_row.get("col_5", 0) != 0 and data_row.get("col_6", 0) != 0:
            data_row["_id"] = generate_hash(data_row)
            spin_history.append(data_row)
    return spin_history


def write_to_mongodb(item, collection):
    try:
        collection.insert_one(item)
    except pymongo.errors.DuplicateKeyError:
        print("Duplicate item found:", item)


def login(driver, username, password):
    driver.get(login_url)

    username_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="email"]'))
    )
    password_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="password"]'))
    )
    login_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'button[type="submit"]'))
    )

    username_input.send_keys(username)
    password_input.send_keys(password)
    login_button.click()

    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, "custom-select"))
    )


uri = "... blank ..."
client = MongoClient(uri)
db = client["mario-scrapper"]
collection = db["data"]
calculation_collection = db["calculation"]

login_url = "https://tracksino.com/login?redirect=%2Fmonopoly?period=1month"
username = "mario.mataga18@gmail.com"
password = "opuzen16"


driver = webdriver.Chrome()

login(driver, username, password)

select_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "custom-select"))
)

select = Select(select_element)
select.select_by_value("100")
time.sleep(5)


def wait_for_table_data_to_load(driver):
    WebDriverWait(driver, 1000).until(
        lambda d: d.find_element(By.CSS_SELECTOR, 'table[aria-busy="false"]')
    )


def get_most_recent_date(collection):
    most_recent_data = collection.find_one(
        sort=[("date", pymongo.DESCENDING)], projection={"date": 1, "_id": 0}
    )
    return most_recent_data["date"] if most_recent_data else None


while True:
    try:
        wait_for_table_data_to_load(driver)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        data = get_spin_history_data(soup)

        for item in data:
            result = write_to_mongodb(item, collection)

        next_button = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@aria-label="Go to next page"]')
            )
        )

        if not next_button.get_attribute("aria-disabled") == "true":
            driver.execute_script("arguments[0].click();", next_button)

        else:
            break

    except Exception as e:
        print("exception", e)
        break


exception encoding with 'idna' codec failed (UnicodeError: label empty or too long), Timeout: 30s, Topology Description: <TopologyDescription id: 65d4442178cfde29c38b99fc, topology_type: Unknown, servers: [<ServerDescription ('... blank ...', 27017) server_type: Unknown, rtt: None, error=UnicodeError("encoding with 'idna' codec failed (UnicodeError: label empty or too long)")>]>
